In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import os
import time
import matplotlib.pyplot as plt

In [24]:
class_name = ['Thumbs Up','Thumbs Down','Peace','Super','You','Call Me','Rock','Love You','High Five','None']

In [26]:
data_path = r'D:\dataset\handgesture'
# for name in class_name:
#     dir = os.path.join(data_path,name)
#     os.makedirs(dir)

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawings = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable= False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [46]:
cap = cv2.VideoCapture(0)
num = 102
j = 9
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence =0.5,) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        # Get the frame and results from Mediapipe
        image, results = mediapipe_detection(frame, holistic)

        if results.right_hand_landmarks:
            
            mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            try:
                if cv2.waitKey(1) & 0xFF == ord('a') and num <=200:
                    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
                    data_path = r'D:\dataset\handgesture'
                    npy_path = os.path.join(data_path,class_name[j],str(num))
                    np.save(npy_path,rh)
                    # print(npy_path)
                    num+=1
                elif num >200:
                    j+=1
                    num = 101
            except:
                break
                
           
        frame = cv2.flip(image, 1)
        cv2.putText(frame,f'collecting {class_name[j]} frame:{str(num)}',(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
        cv2.imshow('sample', frame)

        # Breaking Gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [47]:
cap.release()
cv2.destroyAllWindows()

In [48]:
data,labels = [], []
for label in class_name:
    for num in np.array(os.listdir(os.path.join(data_path, label))):
        res = np.load(os.path.join(data_path, label,num))
        data.append(res)
        labels.append(label)

In [49]:
x = np.array(data)
y = np.array(labels)

In [50]:
class_name 

['Thumbs Up',
 'Thumbs Down',
 'Peace',
 'Super',
 'You',
 'Call Me',
 'Rock',
 'Love You',
 'High Five',
 'None']

In [51]:
x.shape

(2002, 63)

In [52]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(y)

In [53]:
label_names = []
for value in range(10):
    key = le.inverse_transform([value])
    label_names.append(key[0])

In [54]:
label_names = ['Call Me',
 'High Five',
 'Love You',
 'None',
 'Peace',
 'Rock',
 'Super',
 'Thumbs Down',
 'Thumbs Up',
 'You']

In [55]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8,random_state=42)

In [56]:
x_train[0].shape

(63,)

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential()
model.add(Conv1D(40, kernel_size=3, activation='relu', input_shape=(63,1)))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [58]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 61, 40)            160       
                                                                 
 conv1d_5 (Conv1D)           (None, 59, 32)            3872      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 29, 32)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 928)               0         
                                                                 
 dense_4 (Dense)             (None, 200)               185800    
                                                                 
 dense_5 (Dense)             (None, 10)                2010      
                                                      

In [59]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])

In [61]:
model.fit(x_train, y_train,validation_data=(x_test,y_test),epochs=2000)

Epoch 1/2000
13/13 [==============================] - 1s 52ms/step - loss: 0.0819 - acc: 0.9700 - val_loss: 0.3238 - val_acc: 0.8995
Epoch 2/2000
13/13 [==============================] - 1s 46ms/step - loss: 0.0625 - acc: 0.9825 - val_loss: 0.2444 - val_acc: 0.9413
Epoch 3/2000
13/13 [==============================] - 1s 44ms/step - loss: 0.0393 - acc: 0.9900 - val_loss: 0.2492 - val_acc: 0.9401
Epoch 4/2000
13/13 [==============================] - 1s 46ms/step - loss: 0.0409 - acc: 0.9850 - val_loss: 0.2503 - val_acc: 0.9363
Epoch 5/2000
13/13 [==============================] - 1s 44ms/step - loss: 0.0400 - acc: 0.9950 - val_loss: 0.2540 - val_acc: 0.9363
Epoch 6/2000
13/13 [==============================] - 1s 44ms/step - loss: 0.0310 - acc: 0.9900 - val_loss: 0.2742 - val_acc: 0.9332
Epoch 7/2000
13/13 [==============================] - 1s 43ms/step - loss: 0.0293 - acc: 0.9950 - val_loss: 0.2553 - val_acc: 0.9357
Epoch 8/2000
13/13 [==============================] - 1s 43ms/step - 

KeyboardInterrupt: 

In [62]:
model.evaluate(x_test,y_test)

51/51 [==============================] - 1s 9ms/step - loss: 0.2863 - acc: 0.9313


[0.286326140165329, 0.9313358068466187]

In [141]:
yhat = model.predict(x_test)

26/26 [==============================] - 0s 3ms/step


In [147]:
label_names[np.argmax(yhat[1])]

'High Five'

In [148]:
label_names[y_test[1]]

'High Five'

In [65]:
model.save('handgesture_rightv2.h5')

In [63]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence =0.5,) as holistic:
    while cap.isOpened():
            yhat = 3
            ret, frame = cap.read()
    
            # Get the frame and results from Mediapipe
            image, results = mediapipe_detection(frame, holistic)
    
            if results.right_hand_landmarks:
                mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
                x = np.expand_dims(rh,0)
                yhat = np.argmax(model.predict(x))
                print(label_names[yhat])
                
            frame = cv2.flip(image, 1)
            cv2.putText(frame,label_names[yhat],(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
            cv2.imshow('Hand Gesture', frame)
            # Breaking Gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 451ms/step
Thumbs Up
1/1 [==============================] - 0s 45ms/step
Thumbs Up
1/1 [==============================] - 0s 41ms/step
Thumbs Up
1/1 [==============================] - 0s 37ms/step
Thumbs Up
1/1 [==============================] - 0s 49ms/step
Thumbs Up
1/1 [==============================] - 0s 37ms/step
Thumbs Up
1/1 [==============================] - 0s 38ms/step
Thumbs Up
1/1 [==============================] - 0s 37ms/step
Thumbs Up
1/1 [==============================] - 0s 39ms/step
Thumbs Up
1/1 [==============================] - 0s 40ms/step
Thumbs Up
1/1 [==============================] - 0s 40ms/step
Thumbs Up
1/1 [==============================] - 0s 38ms/step
Thumbs Up
1/1 [==============================] - 0s 44ms/step
Thumbs Up
1/1 [==============================] - 0s 43ms/step
Thumbs Up
1/1 [==============================] - 0s 38ms/step
Call Me
1/1 [==============================] - 0s 38ms/step
Call Me
1/1 [======

In [64]:
left_hand_model = tf.keras.models.load_model("lefthandmodel.h5")
model = tf.keras.models.load_model('handgesture_right.h5')

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [19]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence =0.5,) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        # Get the frame and results from Mediapipe
        image, results = mediapipe_detection(frame, holistic)

        if results.left_hand_landmarks:
            
            mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            left_arr = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
            left_arr_reshaped = left_arr.reshape(1, left_arr.shape[0], 1)
            predicted_left = left_hand_model.predict(left_arr_reshaped)
            left_gesture = label_names[np.argmax(predicted_left)]
            print(f"LEFT: {left_gesture}")
            
        if results.right_hand_landmarks:
            
            mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            right_arr = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
            right_arr_reshaped = left_arr.reshape(1, left_arr.shape[0], 1)
            predicted_right = right_hand_model.predict(left_arr_reshaped)
            right_gesture = label_names[np.argmax(predicted_right)]
            print(f"RIGHT: {right_gesture}")
            
                
           
        image = cv2.flip(image, 1)
        if results.left_hand_landmarks:
            cv2.putText(image,left_gesture,(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
        if results.right_hand_landmarks:
            cv2.putText(image,right_gesture,(450,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
        if not results.left_hand_landmarks and results.right_hand_landmarks:
            cv2.putText(image," ",(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
        cv2.imshow('sample', image)

        # Breaking Gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 150ms/step
RIGHT: Peace
1/1 [==============================] - 0s 41ms/step
RIGHT: Peace
1/1 [==============================] - 0s 38ms/step
RIGHT: Peace
1/1 [==============================] - 0s 40ms/step
RIGHT: Peace
1/1 [==============================] - 0s 41ms/step
RIGHT: Peace
1/1 [==============================] - 0s 39ms/step
RIGHT: Peace
1/1 [==============================] - 0s 37ms/step
RIGHT: Peace
1/1 [==============================] - 0s 38ms/step
RIGHT: Peace
1/1 [==============================] - 0s 40ms/step
RIGHT: Peace
1/1 [==============================] - 0s 38ms/step
RIGHT: Peace
1/1 [==============================] - 0s 37ms/step
RIGHT: Peace
1/1 [==============================] - 0s 42ms/step
RIGHT: Peace
1/1 [==============================] - 0s 40ms/step
RIGHT: Peace
1/1 [==============================] - 0s 41ms/step
RIGHT: Peace
1/1 [==============================] - 0s 40ms/step
RIGHT: Peace
1/1 [===================

In [16]:
cap.release()
cv2.destroyAllWindows()